In [1]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [3]:
import pulse_compiler_helper_fns

In [4]:
q.IBMQ.load_account() # Load account from disk

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

/Users/alijavadi/Desktop/ajavadia-terra/qiskit/pulse/channels/pulse_channel_spec.py:87: DeprecationWarning: The PulseChannelSpec is deprecated. Use backend.configuration() instead. The supported methods require some migrations; check out the release notes for the complete details.
  DeprecationWarning)


In [6]:
thetas = np.linspace(0, np.pi/2, 20, endpoint=False)
print(thetas)

[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651]


In [7]:
schedules = []
for theta in thetas:
    circuit = q.QuantumCircuit(2)
    circuit.h(0)
    circuit.h(1)
    circuit.zz_interaction(theta, 0, 1)
    circuit.h(0)
    circuit.h(1)

    decomposed_circuit = circuit.decompose()

    pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
    transpiled_circuit = q.transpile(decomposed_circuit, backend, optimization_level=1)

    schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
    schedule |= measure << schedule.duration
    schedules.append(schedule)

In [8]:
shots = 1000
job = q.execute(schedules, backend=backend, shots=shots)
print('job is %s' % job.job_id())

job is 5dd6d997fa67c10018ceb194


/anaconda3/lib/python3.6/site-packages/ipywidgets/widgets/widget_layout.py:84: DeprecationWarning: Layout properties overflow_x and overflow_y have been deprecated and will be dropped in a future release. Please use the overflow shorthand property instead
  warnings.warn("Layout properties overflow_x and overflow_y have been deprecated and will be dropped in a future release. Please use the overflow shorthand property instead", DeprecationWarning)


In [18]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

In [19]:
result = job.result()

In [20]:
from qiskit.ignis.verification.tomography.data import marginal_counts

In [21]:
counts_list = []
for i in range(len(thetas)):
    counts = marginal_counts(result.get_counts(i), meas_qubits=[0,1])    
    counts_list.append(counts)
    print(counts)

{'00': 954, '01': 28, '10': 14, '11': 4}
{'00': 946, '01': 28, '10': 17, '11': 9}
{'00': 931, '01': 40, '10': 12, '11': 17}
{'00': 874, '01': 67, '10': 20, '11': 39}
{'00': 867, '01': 45, '10': 22, '11': 66}
{'00': 820, '01': 69, '10': 22, '11': 89}
{'00': 777, '01': 89, '10': 30, '11': 104}
{'00': 741, '01': 98, '10': 28, '11': 133}
{'00': 691, '01': 104, '10': 46, '11': 159}
{'00': 618, '01': 143, '10': 45, '11': 194}
{'00': 600, '01': 137, '10': 65, '11': 198}
{'00': 535, '01': 145, '10': 78, '11': 242}
{'00': 517, '01': 183, '10': 84, '11': 216}
{'00': 448, '01': 200, '10': 93, '11': 259}
{'00': 409, '01': 204, '10': 128, '11': 259}
{'00': 364, '01': 241, '10': 148, '11': 247}
{'00': 347, '01': 233, '10': 170, '11': 250}
{'00': 310, '01': 239, '10': 189, '11': 262}
{'00': 273, '01': 259, '10': 211, '11': 257}
{'00': 263, '01': 270, '10': 235, '11': 232}


In [22]:
print(counts_list)

[{'00': 954, '01': 28, '10': 14, '11': 4}, {'00': 946, '01': 28, '10': 17, '11': 9}, {'00': 931, '01': 40, '10': 12, '11': 17}, {'00': 874, '01': 67, '10': 20, '11': 39}, {'00': 867, '01': 45, '10': 22, '11': 66}, {'00': 820, '01': 69, '10': 22, '11': 89}, {'00': 777, '01': 89, '10': 30, '11': 104}, {'00': 741, '01': 98, '10': 28, '11': 133}, {'00': 691, '01': 104, '10': 46, '11': 159}, {'00': 618, '01': 143, '10': 45, '11': 194}, {'00': 600, '01': 137, '10': 65, '11': 198}, {'00': 535, '01': 145, '10': 78, '11': 242}, {'00': 517, '01': 183, '10': 84, '11': 216}, {'00': 448, '01': 200, '10': 93, '11': 259}, {'00': 409, '01': 204, '10': 128, '11': 259}, {'00': 364, '01': 241, '10': 148, '11': 247}, {'00': 347, '01': 233, '10': 170, '11': 250}, {'00': 310, '01': 239, '10': 189, '11': 262}, {'00': 273, '01': 259, '10': 211, '11': 257}, {'00': 263, '01': 270, '10': 235, '11': 232}]
